# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [1]:
import mujoco
import mujoco_viewer
import numpy as np
from numpy.linalg import inv, pinv
import scipy as sp

from utils import rot_err, getSitePose, getSiteVel, getSiteJacobian, getMassMatrix, getBiasTerms, getOSMassMatrix
from autograder import autograder
from sim import Sim

sim = Sim(fname='./a1/xml/a1.xml')

In [ ]:

class OperationalSpaceControl(object):
    def __init__(self, model, data) -> None:
        self._model = model
        self._data  = data 
        self._foot_site_names = [
            "FR_foot", "FL_foot", "RR_foot", "RL_foot"
        ]
        self._kp = 100.
        self._kd = 20.

    def __call__(self, ref_foot_pos):
        mass_matrix = getMassMatrix(self._model, self._data)
        bias        = getBiasTerms(self._model, self._data)
        tau = bias 

        for _foot_name in self._foot_site_names:
            (R_f, p_f) = getSitePose(self._model, self._data, _foot_name)
            jac_f      = getSiteJacobian(self._model, self._data, _foot_name)
            v_f        = jac_f @ self._data.qvel
            (R_d, p_d) = ref_foot_pos[_foot_name]

            err_p   = p_d - p_f
            err_rot = rot_err(R_d, R_f)
            err     = np.concatenate([err_p, err_rot])

            ### calculate the xddot_des in operational space 
            xddot_des = self._kp * err + self._kd *(-v_f)

            ### calculate the operational space mass matrix 
            op_mass_matrix = getOSMassMatrix(mass_matrix, jac_f)

            tau += jac_f.T @ op_mass_matrix @ xddot_des

        return tau
        

In [2]:
t_max = 10000
render = True
if len(sim.model.key_qpos) != 0:
    sim.data.qpos = sim.model.key_qpos
mujoco.mj_step(sim.model, sim.data)
log = []
if render:
    viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
    # viewer.vopt.frame = 3
for t in range(t_max):
    # u = ctrl_method(sim.model, sim.data)
    sim.data.ctrl = np.zeros(sim.model.nu)
    mujoco.mj_step(sim.model, sim.data)
    if render:
        if viewer.is_alive:
            viewer.render()
if render:
    viewer.close()

Pressed ESC
Quitting.


In [3]:





def operational_space_control(model, data):
    # PD control parameters
    kp = 350.
    kd = 50.
    # simulate and render
    (R_ee, p_ee)     = getSitePose(model, data, "ee_site")
    (R_des, p_des)   = getSitePose(model, data, "target_site")
    jac              = getSiteJacobian(model, data, "ee_site")

    ## we can now work directly with errors in the operational space
    err_p   = p_des - p_ee
    err_rot = rot_err(R_des, R_ee)
    err     = np.concatenate([err_p, err_rot])

    ### to compute velocity in operational space, we do the following transform 
    ### this is equivalent to J qdot
    err_dot = jac@data.qvel

    ### calculate the xddot_des in operational space 
    xddot_des = kp * err + kd *(-err_dot)

    ### pull out the dynamic terms 
    mass_matrix = getMassMatrix(model, data)
    bias        = getBiasTerms(model, data)

    ### calculate the operational space mass matrix 
    op_mass_matrix = getOSMassMatrix(mass_matrix, jac)

    ### compute the torque controller with respect to the operational space 
    ### that is J.T (M_x xddot_des) + b
    tau = jac.T @ op_mass_matrix@ xddot_des + bias
    return tau

In [4]:
def simulateCtrl(sim, ctrl_method, render=True, t_max=1000):
    if len(sim.model.key_qpos) != 0:
        sim.data.qpos = sim.model.key_qpos
    mujoco.mj_step(sim.model, sim.data)
    log = []
    if render:
        viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
        viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl_method(sim.model, sim.data)
        sim.data.ctrl = u 
        mujoco.mj_step(sim.model, sim.data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()

In [5]:
simulateCtrl(franka_sim, operational_space_control)

Pressed ESC
Quitting.
